In [1]:
print('hello')

hello


In [ ]:
import vlc
import os
import schedule
import time

def play_mp3(folder_path, start_time, stop_time):
    media_files = [f for f in os.listdir(folder_path) if f.endswith('.mp3')]
    instance = vlc.Instance()
    player = instance.media_list_player_new()
    media_list = instance.media_list_new([os.path.join(folder_path, f) for f in media_files])
    player.set_media_list(media_list)

    # Set the start and stop times for playback
    start_hour, start_minute = map(int, start_time.split(':'))
    stop_hour, stop_minute = map(int, stop_time.split(':'))

    while True:
        # Get the current time
        now = time.localtime()
        current_hour, current_minute = now.tm_hour, now.tm_min

        # Check if it's time to start playback
        if current_hour == start_hour and current_minute == start_minute:
            player.play()

        # Check if it's time to stop playback
        if current_hour == stop_hour and current_minute == stop_minute:
            player.stop()

        # Wait for 1 second before checking again
        time.sleep(1)

def schedule_playback(folder_path, playback_schedule):
    for start_time, stop_time in playback_schedule:
        schedule.every().day.at(start_time).do(play_mp3, folder_path, start_time, stop_time)

    while True:
        schedule.run_pending()
        time.sleep(1)

# Example usage
folder_path = '/path/to/mp3/folder'
playback_schedule = [("06:00", "09:00"), ("12:00", "15:00"), ("18:00", "21:00")]

schedule_playback(folder_path, playback_schedule)
